In [1]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *

import os
import sys
import time
import datetime
import argparse

from PIL import Image

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

In [12]:
parser = argparse.ArgumentParser()
parser.add_argument("-f", dest = 'j_cfile', help = "jupyter config file", default = "file.json", type = str)
parser.add_argument("--image_folder", type=str, default="data/face_samples", help="path to dataset")
parser.add_argument("--model_def", type=str, default="config/face_mask.cfg", help="path to model definition file")
parser.add_argument("--weights_path", type=str, default="checkpoints/yolov3_ckpt_43_0.8610.pth", help="path to weights file")
parser.add_argument("--class_path", type=str, default="data/face_mask/face.names", help="path to class label file")
parser.add_argument("--conf_thres", type=float, default=0.8, help="object confidence threshold")
parser.add_argument("--nms_thres", type=float, default=0.4, help="iou thresshold for non-maximum suppression")
parser.add_argument("--batch_size", type=int, default=1, help="size of the batches")
parser.add_argument("--n_cpu", type=int, default=0, help="number of cpu threads to use during batch generation")
parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
parser.add_argument("--checkpoint_model", type=str, default='checkpoints/yolov3_ckpt_43_0.8610.pth', help="v")
parser.add_argument("--regu_mode", type=int, default=0, help="0:不使用正则化操作;1:使用label smooth;2:使用dropblock;3:两者均使用")
parser.add_argument("--active_mode", type=int, default=1, help="0:原始激活方式;1:Swish;2:Mish")
opt = parser.parse_args()
print(opt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("output", exist_ok=True)

# Set up model
model = Darknet(opt.model_def, img_size=opt.img_size, regu_mode=opt.regu_mode, active_mode=opt.active_mode).to(device)

if opt.weights_path.endswith(".weights"):
    # Load darknet weights
    model.load_darknet_weights(opt.weights_path)
else:
    # Load checkpoint weights
    model.load_state_dict(torch.load(opt.weights_path))

model.eval()  # Set in evaluation mode

dataloader = DataLoader(
    ImageFolder(opt.image_folder, img_size=opt.img_size),
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_cpu,
)

classes = load_classes(opt.class_path)  # Extracts class labels from file

Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

imgs = []  # Stores image paths
img_detections = []  # Stores detections for each image index

print("\nPerforming object detection:")
prev_time = time.time()
t = 0
num = 0
for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
    # Configure input
    input_imgs = Variable(input_imgs.type(Tensor))

    # Get detections
    with torch.no_grad():
        detections = model(input_imgs)
        detections = non_max_suppression(detections, opt.conf_thres, opt.nms_thres)

    # Log progress
    current_time = time.time()
    inference_time = datetime.timedelta(seconds=current_time - prev_time)
    t += (current_time - prev_time)
    num += 1
    prev_time = current_time
    print("\t+ Batch %d, Inference Time: %s" % (batch_i, inference_time))

    # Save image and detections
    imgs.extend(img_paths)
    img_detections.extend(detections)
print("平均inference时间:{}s".format(t/num))
# Bounding-box colors
cmap = plt.get_cmap("tab20b")
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

print("\nSaving images:")
# Iterate through images and save plot of detections
for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):

    print("(%d) Image: '%s'" % (img_i, path))

    # Create plot
    img = np.array(Image.open(path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Draw bounding boxes and labels of detections
    if detections is not None:
        # Rescale boxes to original image
        detections = rescale_boxes(detections, opt.img_size, img.shape[:2])
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        bbox_colors = random.sample(colors, n_cls_preds)
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

            print("\t+ Label: %s, Conf: %.5f" % (classes[int(cls_pred)], cls_conf.item()))

            box_w = x2 - x1
            box_h = y2 - y1

            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            # Create a Rectangle patch
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor="none")
            # Add the bbox to the plot
            ax.add_patch(bbox)
            # Add label
            plt.text(
                x1,
                y1,
                s=classes[int(cls_pred)],
                color="white",
                verticalalignment="top",
                bbox={"color": color, "pad": 0},
            )

    # Save generated image with detections
    plt.axis("off")
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    filename = path.split("/")[-1].split(".")[0]
    active_mode = 'act'+ str(opt.active_mode)
    plt.savefig(f"output/{filename}_{active_mode}.png", bbox_inches="tight", pad_inches=0.0)
    plt.close()


Namespace(active_mode=1, batch_size=1, checkpoint_model='checkpoints/yolov3_ckpt_43_0.8610.pth', class_path='data/face_mask/face.names', conf_thres=0.8, image_folder='data/face_samples', img_size=416, j_cfile='/home/aim/.local/share/jupyter/runtime/kernel-345f58c2-4abf-400d-9757-823a38fc8447.json', model_def='config/face_mask.cfg', n_cpu=0, nms_thres=0.4, regu_mode=0, weights_path='checkpoints/yolov3_ckpt_43_0.8610.pth')

Performing object detection:
	+ Batch 0, Inference Time: 0:00:00.122173
	+ Batch 1, Inference Time: 0:00:00.078708
	+ Batch 2, Inference Time: 0:00:00.073160
	+ Batch 3, Inference Time: 0:00:00.096383
	+ Batch 4, Inference Time: 0:00:00.113029
	+ Batch 5, Inference Time: 0:00:00.055083
	+ Batch 6, Inference Time: 0:00:00.055898
	+ Batch 7, Inference Time: 0:00:00.063616
	+ Batch 8, Inference Time: 0:00:00.057884
	+ Batch 9, Inference Time: 0:00:00.089661
	+ Batch 10, Inference Time: 0:00:00.058340
	+ Batch 11, Inference Time: 0:00:00.086070
	+ Batch 12, Inference Time

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>